In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Transaction Type Consolidation Demo

Attributes
----------
transactions
transaction types
portfolios
instruments
"""

toggle_code("Hide docstring")

# Transaction Type Consolidation

## Table of contents

- 1. [Overview](#1.-Overview)
- 2. [Setup](#2.-Setup)
- 3. [Transaction Configuration](#3.-Transaction-Configuration)
   * [3.1 Transaction Type Overview](#3.1-Transaction-Type-Overview)
   * [3.2 Transaction Type Definitions](#3.2-Transaction-Type-Definitions)
- 4. [Load Data](#4.-Load-Data)
    * [4.1 Portfolios](#4.1-Portfolios)
    * [4.2 Portfolio Group](#4.2-Portfolio-Group)
    * [4.3 Instruments](#4.3-Instruments)
    * [4.4 Transactions](#4.4-Transactions)
- 5. [View Output Transactions](#5.-View-Output-Transactions)
    * [5.1 Output Transactions of EnergyFundGlobalGroup](#5.1-Output-Transactions-of-EnergyFundGlobalGroup)
    * [5.2 Holdings view for EnergyFundUS](#5.2-Holdings-view-for-EnergyFundUS)

# 1. Overview

In this Notebook, we'll look at how LUSID can support the consolidation of multiple transaction type taxonomies from external sources into a single convention across all portfolios.

To show this, we'll look at three energy focused portfolios:

- EnergyFundUS
- EnergyFundAmericasExUS
- EnergyFundEurope

We'll assume EnergyFundUS is a directly managed portfolio while management of the two other funds is outsourced to an external provider. Furthermore, we'll assume that EnergyFundAmericasExUS and EnergyFundEurope are custodied at 'US Custody Bank' and 'French Custody Bank' respectively.

# 2. Setup

We first initialize our various Python libraries, objects, and datasets required to construct our examples:

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
import json
import pytz
import uuid
from datetime import datetime
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from flatten_json import flatten

import os
import pandas as pd
import math

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

#Load LUSID API Components
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
portfolio_groups_api = api_factory.build(lusid.api.PortfolioGroupsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
txn_api = api_factory.build(lusid.api.TransactionConfigurationApi)

# Set Global Scope
global_scope = "ibor"

# Portfolio Creation Date
portfolio_creation_date="2022-01-01"

# Transaction Portfolios
transaction_portfolios = ["EnergyFundUS", "EnergyFundAmericasExUS", "EnergyFundEurope"]

# Portfolio Group
portfolio_group_code = "EnergyFundGlobalGroup"

# Load Requisite Data
transaction_data = pd.read_excel("data/transaction_type_consolidation_data.xlsx", sheet_name="transactions")
instrument_data = pd.read_excel("data/transaction_type_consolidation_data.xlsx", sheet_name="instruments")

# 3. Transaction Configuration

## 3.1 Transaction Type Overview

As mentioned, we want to consolidate multiple transaction type taxonomies into a single standard within LUSID. In our example, we'll look at a 'security purchase' type of transaction. Our OMS will call this *Purchase*, while US Custody Bank and French Custody Bank will call these *BuyL* and *101* respectively. 

What we ultimately then want to do is harmonize these into a standard transaction type we call *Buy*. This construction is illustrated below:

![Init](img/TaxonomyMapping.PNG)

## 3.2 Transaction Type Definitions

To implement the above, we'll make use of transaction type aliases in LUSID. 

We'll need to define four aliases in our transaction type definition:

- **Purchase**: The transaction type name used by our enterprise OMS
- **BuyL**: The transaction type name used by US Custody Bank
- **101**: The transaction type name used by French Custody Bank
- **Buy**: The effective transaction type name we want to see in LUSID

Because the economics of these transaction types are the same, each alias will share the same movements. We thus configure our transaction as follows:

In [ ]:
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="Purchase",
                description="Purchase transaction type from enterprise OMS",
                transaction_class="CashSecurityPurchase",
                transaction_group="OMS",
                source="OMS",
                transaction_roles="AllRoles",
            ),
            models.TransactionConfigurationTypeAlias(
                type="BuyL",
                description="Buy transaction type from US custody bank feed",
                transaction_class="CashSecurityPurchase",
                transaction_group="USCustodyBank",
                source="USCustodyBank",
                transaction_roles="AllRoles",
            ),
            models.TransactionConfigurationTypeAlias(
                type="101",
                description="Buy transaction type from French custody bank feed",
                transaction_class="CashSecurityPurchase",
                transaction_group="FrenchCustodyBank",
                source="FrenchCustodyBank",
                transaction_roles="AllRoles",
            ),
            models.TransactionConfigurationTypeAlias(
                type="Buy",
                description="IBOR Buy Transaction",
                transaction_class="CashSecurityPurchase",
                transaction_group="default",
                source="default",
                transaction_roles="AllRoles",
            )              
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
            ),models.TransactionConfigurationMovementDataRequest(
                movement_types="CashReceivable",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

print("Finished buy related transaction configuration Upsert!")

What's critical to this setup is that we set the `source` to 'default' for the *Buy* transaction alias. This tells LUSID that if any of the other three transaction types have been loaded into LUSID with the same `transaction_class` and `transaction_roles` value as the default source alias, our 'effective' transaction type name will be *Buy*. Note that LUSID maintains the original input transaction type which is accessible via the `Transaction/default/TxnInputType` property returned from a transaction extract. We show this in section [5.1](#5.1-Output-Transactions-of-EnergyFundGlobalGroup) below.

For more details on roles, classes, and aliases, please see the following articles [here](https://support.lusid.com/knowledgebase/article/KA-01876/en-us) and [here](https://support.lusid.com/knowledgebase/article/KA-01872/).

# 4. Load Data

Now that we've defined our transaction types, we'll load some sample portfolio, instrument, and transaction data into LUSID.
## 4.1 Portfolios

In [ ]:
# Create our Transaction Portfolios
def load_txn_portfolio(portfolio_code):
    try:
        transaction_portfolio_api.create_portfolio(
            scope=global_scope,
            create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
                display_name=portfolio_code,
                code=portfolio_code,
                base_currency="USD",
                created=portfolio_creation_date,
                instrument_scopes=[global_scope]
            ),
        )
        print("Portfolio: " + portfolio_code + " loaded!")

    except lusid.ApiException as e:
        print(json.loads(e.body)["title"])

for portfolio in transaction_portfolios:
    load_txn_portfolio(portfolio)

## 4.2 Portfolio Group

We'll group our three funds into a Portfolio Group to more easily view all transactions together.

In [ ]:
#Create a group portfolio for our three energy funds
portfolio_resource_ids = [
            models.ResourceId(
                scope=global_scope,
                code=portfolio_code) for portfolio_code in transaction_portfolios]

try:
    group_request = models.CreatePortfolioGroupRequest(
        code=portfolio_group_code,
        display_name=portfolio_group_code,
        values=portfolio_resource_ids,
        sub_groups=None,
        description=None,
        created=portfolio_creation_date)
    
    portfolio_group = portfolio_groups_api.create_portfolio_group(
        scope=global_scope,
        create_portfolio_group_request=group_request)    
    print("Portfolio: " + portfolio_group_code + " created!")

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

## 4.3 Instruments

In [ ]:
# Load publicly listed equities
def load_equity(data):
          
    client_internal = "Instrument/default/ClientInternal"
    
    equity = models.Equity(
        instrument_type="Equity",
        dom_ccy=data["currency"],
    )

    equity_definition = models.InstrumentDefinition(
        name=data["name"],
        identifiers={"ClientInternal": models.InstrumentIdValue(data["client_internal"]),
                     "Ticker": models.InstrumentIdValue(data["ticker"])
                    },
        definition=equity,
        properties=[],        
    )

    # upsert the instrument
    upsert_request = {client_internal: equity_definition}
    upsert_response = instruments_api.upsert_instruments(scope=global_scope, request_body=upsert_request)
    equity_luid = upsert_response.values[client_internal].lusid_instrument_id

for index, row in instrument_data.iterrows():

    load_equity(row)

print ("Instruments Upserted!")

In [7]:
instrument_data

,name,client_internal,ticker,currency
0,Exxon Mobil Corporation,eq_us_XOM,XOM,USD
1,Chevron Corporation,eq_us_CVX,CVX,USD
2,Chesapeake Energy Corporation,eq_us_CHK,CHK,USD
3,Occidental Petroleum Corporation,eq_us_OXY,OXY,USD
4,Suncor Energy,eq_ca_SU,SU,CAD
5,Imperial Oil,eq_ca_IMO,IMO,CAD
6,Petróleo Brasileiro S.A.,eq_br_PBR,PBR,USD
7,TotalEnergies SE,eq_fr_TTE,TTE,EUR
8,Engie SA,eq_fr_ENGI,ENGI,EUR
9,Repsol S.A.,eq_sp_REP,REP,EUR


## 4.4 Transactions

In [ ]:
# Load Transactions
for index, row in transaction_data.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_internal"] }
    
    if isinstance(row["client_internal"], float):
        primary_instrument_identifier = { "Instrument/default/Currency": row["currency"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=global_scope,
        code=row['portfolio'],
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["settle_date"],
                units=row["quantity"],
                source=row["txn_source"],
                transaction_price=models.TransactionPrice(
                    price=row["txn_price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=row["currency"]
                ),
            )
        ],
    )

print("Done!")

In [9]:
transaction_data

,txn_id,portfolio,txn_type,client_internal,currency,txn_source,trade_date,settle_date,quantity,txn_price,total_consideration
0,txnid_OMS0001,EnergyFundUS,Purchase,eq_us_XOM,USD,OMS,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,9500,70,665000
1,txnid_OMS0002,EnergyFundUS,Purchase,eq_us_CVX,USD,OMS,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,5000,125,625000
2,txnid_OMS0003,EnergyFundUS,Purchase,eq_us_CHK,USD,OMS,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,9250,70,647500
3,txnid_OMS0004,EnergyFundUS,Purchase,eq_us_OXY,USD,OMS,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,20000,33,660000
4,txnid_SSCust0001,EnergyFundAmericasExUS,BuyL,eq_ca_SU,CAD,USCustodyBank,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,15000,33,495000
5,txnid_SSCust0002,EnergyFundAmericasExUS,BuyL,eq_ca_IMO,CAD,USCustodyBank,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,10000,50,500000
6,txnid_SSCust0003,EnergyFundAmericasExUS,BuyL,eq_br_PBR,USD,USCustodyBank,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,45000,11,495000
7,txnid_BNPCust0001,EnergyFundEurope,101,eq_fr_TTE,EUR,FrenchCustodyBank,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,7500,45,337500
8,txnid_BNPCust0002,EnergyFundEurope,101,eq_fr_ENGI,EUR,FrenchCustodyBank,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,25000,13,325000
9,txnid_BNPCust0003,EnergyFundEurope,101,eq_sp_REP,EUR,FrenchCustodyBank,2022-01-05T00:00:00Z,2022-01-07T00:00:00Z,33000,10,330000


# 5. View Output Transactions

## 5.1 Output Transactions of EnergyFundGlobalGroup

With our requisite data now in place, we'll call `build_transactions_for_portfolio_group()` to view all transactions within a given time horizon. 

In [10]:
transaction_query_parameters=models.TransactionQueryParameters(start_date="2022-01-01", end_date="2022-02-01")
input_transactions = lusid_response_to_data_frame(
    portfolio_groups_api.build_transactions_for_portfolio_group(
        scope=global_scope, 
        code='EnergyFundGlobalGroup',
        transaction_query_parameters=transaction_query_parameters)
)

rename_cols = {}
rename_cols[f"instrument_identifiers.Instrument/default/ClientInternal"] = "client_internal"
rename_cols[f"transaction_price.price"] = "price"
rename_cols[f"properties.Transaction/default/TxnInputType.value.label_value"] = "input_type"

input_transactions.rename(
    columns=rename_cols,
    inplace=True,
)

columnsReordered = ['transaction_id', 'type', 'input_type','client_internal', 'transaction_date', 'settlement_date', 'price', 'units']
input_transactions = input_transactions.reindex(columns=columnsReordered)

input_transactions

,transaction_id,type,input_type,client_internal,transaction_date,settlement_date,price,units
0,txnid_OMS0001,Buy,Purchase,eq_us_XOM,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,70.00,"9,500.00"
1,txnid_OMS0002,Buy,Purchase,eq_us_CVX,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,125.00,"5,000.00"
2,txnid_OMS0003,Buy,Purchase,eq_us_CHK,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,70.00,"9,250.00"
3,txnid_OMS0004,Buy,Purchase,eq_us_OXY,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,33.00,"20,000.00"
4,txnid_SSCust0001,Buy,BuyL,eq_ca_SU,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,33.00,"15,000.00"
5,txnid_SSCust0002,Buy,BuyL,eq_ca_IMO,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,50.00,"10,000.00"
6,txnid_SSCust0003,Buy,BuyL,eq_br_PBR,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,11.00,"45,000.00"
7,txnid_BNPCust0001,Buy,101,eq_fr_TTE,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,45.00,"7,500.00"
8,txnid_BNPCust0002,Buy,101,eq_fr_ENGI,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,13.00,"25,000.00"
9,txnid_BNPCust0003,Buy,101,eq_sp_REP,2022-01-05 00:00:00+00:00,2022-01-07 00:00:00+00:00,10.00,"33,000.00"


Importantly, you'll notice the above results contain two columns: type and input_type.

- **type** shows the effective 'harmonized' transaction type alias for a security purchase
- **input_type** shows the source transaction type alias that maps to the external upstream source system

## 5.2 Holdings view for EnergyFundUS

We'll now look at the holdings of EnergyFundUS via `get_holdings()`. We'll call this for January 6, 2022 to include unsettled transactions.

In [11]:
df=lusid_response_to_data_frame(transaction_portfolio_api.get_holdings(
    scope='ibor',
    code='EnergyFundUS',
    effective_at='2022-01-06T00:00:00Z'))

columnsReordered = ['instrument_uid', 'units', 'holding_type_name', 'cost.currency','cost.amount', 'transaction.transaction_id', 'transaction.type']
df = df.reindex(columns=columnsReordered)
df

,instrument_uid,units,holding_type_name,cost.currency,cost.amount,transaction.transaction_id,transaction.type
0,LUID_00003DCY,"9,500.00",Position,USD,"665,000.00",NaN,NaN
1,LUID_00003DCZ,"5,000.00",Position,USD,"625,000.00",NaN,NaN
2,LUID_00003DLA,"9,250.00",Position,USD,"647,500.00",NaN,NaN
3,LUID_00003DLB,"20,000.00",Position,USD,"660,000.00",NaN,NaN
4,CCY_USD,"-665,000.00",Receivable,USD,"-665,000.00",txnid_OMS0001,Buy
5,CCY_USD,"-625,000.00",Receivable,USD,"-625,000.00",txnid_OMS0002,Buy
6,CCY_USD,"-647,500.00",Receivable,USD,"-647,500.00",txnid_OMS0003,Buy
7,CCY_USD,"-660,000.00",Receivable,USD,"-660,000.00",txnid_OMS0004,Buy


We can see the `transaction.type` column returns *Buy*